# Create weights for each sample, based on the instrument distribution in the dataset

Data pipeline: 

Questions
1. train and test splits are provided. should we balance the entire dataset and rechoose train and test (may disturb the pre-trained stuff) or we balance within train and test separately?
- If pytorch, using a sampler is better. Allows per epoch random sampling according to the weights. But not sure if they can handle multi label multi instance problem?
2. should cutmix be done as a part of balancing? other option being do when loading in data.
3. should cutmix be used even? other augmentation techniques? no augmentation needed really?



In [2]:
%cd openmic-2018/

/home/studio-lab-user/sagemaker-studiolab-notebooks/ESP3201-Instrument-indentification/openmic-2018


In [3]:
import json
import os
import numpy as np
!pip install pandas
import pandas as pd

In [4]:
data = np.load("openmic-2018.npz", allow_pickle = True)
X, Y_true, Y_mask, sample_key = data['X'], data['Y_true'], data['Y_mask'], data['sample_key']
print(X.shape, Y_true.shape, Y_mask.shape, sample_key.shape)

(20000, 10, 128) (20000, 20) (20000, 20) (20000,)


## What's included in the data?
- X: 20000 10 128 array of VGGish features
    - First index (0..19999) corresponds to the sample key
    - Second index (0..9) corresponds to the time within the clip (each time slice is 960 ms long)
    - Third index (0..127) corresponds to the VGGish features at each point in the 10sec clip
    - Example X[40, 8] is the 128-dimensional feature vector for the 9th time slice in the 41st example
- Y_true: 20000 20 array of true* label probabilities
    - First index corresponds to sample key, as above
    - Second index corresponds to the label class (accordion, ..., voice)
    - Example: Y[40, 4] indicates the confidence that example #41 contains the 5th instrument
- Y_mask: 20000 * 20 binary mask values (not sure what this is)
    - First index corresponds to sample key
    - Second index corresponds to the label class
    - Example: Y[40, 4] indicates *whether or not we have observations* for the 5th instrument for example #41
- sample_key: 20000 array of sample key strings
    - Example: sample_key[40] is the sample key for example #41


In [5]:
# PROLLY DONT NEED THIS
# Load training split csv file
with open("partitions/split01_train.csv") as f:
    train_IDs = f.readlines()
    train_IDs = np.array([ID.strip() for ID in train_IDs])
print(train_IDs.shape)

# Load test split csv file
with open("partitions/split01_test.csv") as f:
    test_IDs = f.readlines()
    test_IDs = np.array([ID.strip() for ID in test_IDs])
print(test_IDs.shape)

# Get the training and testing split data into np arrays
# Creates an array, which contains the index of the sample in data.  
train_index = np.array([i for i in range(20000) if sample_key[i] in train_IDs])
test_index = np.array([i for i in range(20000) if sample_key[i] in test_IDs])

print(train_index.shape)
print(test_index.shape)

(14915,)
(5085,)
(14915,)
(5085,)


In [6]:
with open('class-map.json', 'r') as f:
    class_map = json.load(f)
print(class_map)

{'accordion': 0, 'banjo': 1, 'bass': 2, 'cello': 3, 'clarinet': 4, 'cymbals': 5, 'drums': 6, 'flute': 7, 'guitar': 8, 'mallet_percussion': 9, 'mandolin': 10, 'organ': 11, 'piano': 12, 'saxophone': 13, 'synthesizer': 14, 'trombone': 15, 'trumpet': 16, 'ukulele': 17, 'violin': 18, 'voice': 19}


In [9]:
# distribution of classes in the dataset 
def get_label(Y_true, idx, inst):
    label = Y_true[idx, inst]
    label = label > 0.5
    return label
indices = list(range(0,20000))
inst_freq = {}
#inefficient but since one time, dont care
for inst in list(class_map.values()):
    label_to_count = {}
    # print(inst)
    for idx in indices:
        # print(idx)
        label = get_label(Y_true, idx, inst)
        if label in label_to_count:
            label_to_count[label] += 1
        else:
            label_to_count[label] = 1
    # print(label_to_count)
    inst_freq[inst] = label_to_count[True]

print(inst_freq)    
weights = [1.0 / i for i in list(inst_freq.values())] 
print(weights)
    # weight for each sample
    # weights = [1.0 / label_to_count[get_label(data, idx, inst)] for idx in indices]
    # weights = torch.DoubleTensor(weights)
    # print(weights)

# def __iter__(self):
#     return (self.indices[i] for i in torch.multinomial(
#         self.weights, self.num_samples, replacement=True))

# def __len__(self):
#     return self.num_samples

{0: 489, 1: 732, 2: 549, 3: 824, 4: 533, 5: 1111, 6: 1106, 7: 647, 8: 1138, 9: 733, 10: 845, 11: 603, 12: 1170, 13: 1135, 14: 1091, 15: 863, 16: 1146, 17: 738, 18: 1173, 19: 988}
0.024660382028429725


In [ ]:
print(Y_mask[1])


[False False False False False False False False False False False False
 False  True False False  True False False  True]


In [28]:
print(Y_true[1])


[0.5     0.5     0.5     0.5     0.5     0.5     0.5     0.5     0.5
 0.5     0.5     0.5     0.5     0.14705 0.5     0.5     0.      0.5
 0.5     1.     ]
